# Deep Neural Network

### Image Mode

In [21]:
IMAGE_MODE = 1      #1 for colour, 0 for grayscale, unchanged for -1

### Datasets locations:

In [22]:
train_csv = "..\\Datasets\\Resized_data_50_50\\train.csv"
test_csv = "..\\Datasets\\Resized_data_50_50\\test.csv"

N_CLASSES = 2

## Imports

In [23]:
import tensorflow as tf

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import cv2

## Loading the dataset

In [24]:
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

In [25]:
train_df.head()

,Image_Path,Parasitized
0,..\Datasets\Resized_data_50_50\Parasitized\50x...,1.0
1,..\Datasets\Resized_data_50_50\Parasitized\50x...,1.0
2,..\Datasets\Resized_data_50_50\Uninfected\50x5...,0.0
3,..\Datasets\Resized_data_50_50\Parasitized\50x...,1.0
4,..\Datasets\Resized_data_50_50\Parasitized\50x...,1.0


In [26]:
train_df.dtypes

Image_Path      object
Parasitized    float64
dtype: object

## Data and Labels

In [27]:
train_x = train_df['Image_Path'].to_numpy()
train_y = train_df['Parasitized'].to_numpy()

test_x  = test_df['Image_Path'].to_numpy()
test_y  = test_df['Parasitized'].to_numpy()

#### Load images from paths

In [28]:
def load_images(path_arr):
    '''Reads and loads images into a numpy array
    Returns: a numpy array'''
    arr = []
    for path in path_arr:
        arr.append(cv2.imread(path, IMAGE_MODE))
    
    return np.array(arr)


In [29]:
train_x = load_images(train_x)
test_x = load_images(test_x)

#### Checking shape

In [30]:
print(train_x.shape, test_x.shape, sep = '\n')

(24802, 50, 50, 3)
(2756, 50, 50, 3)


## DNN

In [31]:
# batch sizes
TRAIN_BATCH_SIZE = 128
TEST_BATCH_SIZE  = 64

In [32]:
train_x = train_x/255.0
test_x = test_x/255.0

train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y))
test_ds  = tf.data.Dataset.from_tensor_slices((test_x, test_y))

train_ds = train_ds.batch(TRAIN_BATCH_SIZE)
test_ds  = test_ds.batch(TEST_BATCH_SIZE)

In [33]:
height, width = train_x.shape[1], train_x.shape[2]
dims = len(train_x.shape)-1

In [34]:
type(train_ds)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [35]:
input_shape = list(train_x.shape[1:])
if(len(input_shape) == 2): input_shape.append(1)

input_shape

[50, 50, 3]

In [69]:
dnn_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding = 'same', activation = 'relu', input_shape = input_shape),
    tf.keras.layers.MaxPool2D((3,3), padding = 'same'),

    tf.keras.layers.Conv2D(8, (3,3), padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPool2D((3,3), padding = 'same'),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(N_CLASSES, activation = 'sigmoid')
])

dnn_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0003),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

dnn_model.fit(train_ds, batch_size = TRAIN_BATCH_SIZE, shuffle = True, epochs = 50, validation_data = test_ds, validation_batch_size= TEST_BATCH_SIZE, callbacks = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 3))

Epoch 1/50
194/194 [==============================] - 2s 9ms/step - loss: 0.6119 - accuracy: 0.6690 - val_loss: 0.5618 - val_accuracy: 0.7184
Epoch 2/50
194/194 [==============================] - 1s 7ms/step - loss: 0.5163 - accuracy: 0.7515 - val_loss: 0.4713 - val_accuracy: 0.7772
Epoch 3/50
194/194 [==============================] - 1s 7ms/step - loss: 0.3935 - accuracy: 0.8282 - val_loss: 0.3315 - val_accuracy: 0.8494
Epoch 4/50
194/194 [==============================] - 1s 7ms/step - loss: 0.2563 - accuracy: 0.9055 - val_loss: 0.2316 - val_accuracy: 0.9115
Epoch 5/50
194/194 [==============================] - 1s 7ms/step - loss: 0.1960 - accuracy: 0.9324 - val_loss: 0.1899 - val_accuracy: 0.9307
Epoch 6/50
194/194 [==============================] - 1s 7ms/step - loss: 0.1752 - accuracy: 0.9410 - val_loss: 0.1845 - val_accuracy: 0.9329
Epoch 7/50
194/194 [==============================] - 1s 7ms/step - loss: 0.1623 - accuracy: 0.9471 - val_loss: 0.1746 - val_accuracy: 0.9343
Epoch 